In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/M2/Vision/ffhq.zip' # original
!unzip '/content/drive/MyDrive/M2/Vision/stylegan_ffhq.zip' # forged
# !unzip '/content/drive/MyDrive/M2/Vision/stylegan_celeba.zip' # forged

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: stylegan_ffhq/train/F_SyFQ_09989.png  
  inflating: stylegan_ffhq/train/F_SyFQ_00002.png  
  inflating: stylegan_ffhq/train/F_SyFQ_00005.png  
  inflating: stylegan_ffhq/train/F_SyFQ_00016.png  
  inflating: stylegan_ffhq/train/F_SyFQ_00035.png  
  inflating: stylegan_ffhq/train/F_SyFQ_00076.png  
  inflating: stylegan_ffhq/train/F_SyFQ_00082.png  
  inflating: stylegan_ffhq/train/F_SyFQ_00093.png  
  inflating: stylegan_ffhq/train/F_SyFQ_00098.png  
  inflating: stylegan_ffhq/train/F_SyFQ_00116.png  
  inflating: stylegan_ffhq/train/F_SyFQ_00135.png  
  inflating: stylegan_ffhq/train/F_SyFQ_00141.png  
  inflating: stylegan_ffhq/train/F_SyFQ_00142.png  
  inflating: stylegan_ffhq/train/F_SyFQ_00143.png  
  inflating: stylegan_ffhq/train/F_SyFQ_00156.png  
  inflating: stylegan_ffhq/train/F_SyFQ_00180.png  
  inflating: stylegan_ffhq/train/F_SyFQ_00192.png  
  inflating: stylegan_ffhq/train/F_SyFQ_00201.png  
  inflating: stylegan_ffhq/t

In [1]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 472.8 kB/s eta 0:00:00


In [2]:
!pip install progress

  Preparing metadata (setup.py) ... done
  Created wheel for progress: filename=progress-1.6-py3-none-any.whl size=9614 sha256=a9a8e9688b55b459f606ac433931e6dfbc1704cefb9bfb436db7fef9b2ffed78
  Stored in directory: /root/.cache/pip/wheels/a2/68/5f/c339b20a41659d856c93ccdce6a33095493eb82c3964aac5a1
Successfully built progress


In [3]:
import os
import random
import math
import yaml
import argparse
import collections
from progress.bar import ChargingBar

import numpy as np
from PIL import Image
import torch
from torch.optim import lr_scheduler

In [4]:
!ls

efficientnet  efficient-vit  sample_data


In [5]:
%cd /content/efficientnet/
from model import EfficientNet

/content/efficientnet


In [6]:
%cd /content/efficient-vit/
from dataset import DeepFakesDataset
from efficient_vit import EfficientViT
from utils import get_method, check_correct, resize, shuffle_dataset, create_label, get_n_params

/content/efficient-vit


In [7]:
!pwd

/content/efficient-vit


In [8]:
%cd /content

/content


In [11]:
# ffhq : R_FFHQ_11000.png
# stylegan_celeba : F_SyCA_11000.png
# stylegan_ffhq : F_SyFQ_10000.png
def getDataset(dataSrcPath:str) -> tuple :
    tr = {'ffhq':None, 'stylegan_ffhq':None, 'stylegan_celeba':None}
    vl = {'ffhq':None, 'stylegan_ffhq':None, 'stylegan_celeba':None}
    ts = {'ffhq':None, 'stylegan_ffhq':None, 'stylegan_celeba':None}

    for k in tr.keys():
        try:
          # images
          tr[k] = [os.path.join(dataSrcPath, k, 'train', img) for img in os.listdir(os.path.join(dataSrcPath, k, 'train'))]
          vl[k] = [os.path.join(dataSrcPath, k, 'validation', img) for img in os.listdir(os.path.join(dataSrcPath, k, 'validation'))]
          ts[k] = [os.path.join(dataSrcPath, k, 'test', img) for img in os.listdir(os.path.join(dataSrcPath, k, 'test'))]
        except:
          print(f"data {k} does not exist")
    return tr, vl, ts

In [12]:
with open('./efficient-vit/configs/configuration.yaml', 'r') as ymlfile:
    config = yaml.safe_load(ymlfile)

In [ ]:
# dataset
DATAPATH = '/content/' #opt.trainpath
tr, vl, ts = getDataset(DATAPATH)
trainset = tr['ffhq'] + tr['stylegan_ffhq']
validset = vl['ffhq'] + vl['stylegan_ffhq']
testset = ts['ffhq'] + ts['stylegan_ffhq']


trainset = shuffle_dataset(trainset)
validset = shuffle_dataset(validset)
testset = shuffle_dataset(testset)
# create labels
trlb = create_label(trainset, DFcheckStr='F_SyFQ')
vllb = create_label(validset, DFcheckStr='F_SyFQ')
tslb = create_label(testset, DFcheckStr='F_SyFQ')

trainset, validset, testset = (trainset, trlb), (validset, vllb), (testset, tslb)

train_samples = len(trainset[0])
validation_samples = len(validset[0])

# Print some statistics
print("Train images:", train_samples, "Validation images:", validation_samples)
print("__TRAINING STATS__")
train_counters = collections.Counter(image for image in trainset[1])
print(train_counters)

class_weights = train_counters[0] / train_counters[1]
print("Weights", class_weights)

print("__VALIDATION STATS__")
val_counters = collections.Counter(image for image in validset[1])
print(val_counters)

print("__TEST STATS__")
test_counters = collections.Counter(image for image in testset[1])
print(test_counters)

data stylegan_celeba does not exist
Train images: 19999 Validation images: 1999
__TRAINING STATS__
Counter({1: 10000, 0: 9999})
Weights 0.9999
__VALIDATION STATS__
Counter({0: 1000, 1: 999})
__TEST STATS__
Counter({1: 9000, 0: 8997})


In [21]:
del model

In [22]:
# model
channels = 1280 #1280

models = 'efficientvit' # 'efficientnet' || 'vit'

# model = EfficientViT(config=config, channels=channels, selected_efficient_net=0, vit=False)
if models == 'efficientvit':
    model = EfficientViT(config=config, channels=channels, selected_efficient_net=1)
elif models == 'efficientnet':
    model = EfficientViT(config=config, channels=channels, selected_efficient_net=1, vit=False)
model.train()

optimizer = torch.optim.SGD(model.parameters(), lr=config['training']['lr'],
                            weight_decay=config['training']['weight-decay'])
scheduler = lr_scheduler.StepLR(optimizer, step_size=config['training']['step-size'],
                                gamma=config['training']['gamma'])
# resume
# starting_epoch = 0
# checkpath = '/content/drive/MyDrive/M2/Vision/efficientnetB1_checkpoint22_test'
# if os.path.exists(checkpath):
#     model.load_state_dict(torch.load(checkpath))
#     starting_epoch = int(checkpath.split("checkpoint")[-1].split("_")[0]) + 1  # The checkpoint's file name format should be "checkpoint_EPOCH"
#     print("checkpoint loaded.")
# else:
#     print("No checkpoint loaded.")

print("Model Parameters:", get_n_params(model))
loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([class_weights]))

Loaded pretrained weights for efficientnet-b1
Efficientnet-b1 will be using.
Vision Transformer will be attached to the efficientnet.
Model Parameters: 111953417


In [ ]:
train_labels = trainset[1]
trainset = DeepFakesDataset(trainset[0], trainset[1], config['model']['image-size'])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True, sampler=None, #config['training']['bs']
                                  batch_sampler=None, num_workers=2, collate_fn=None,
                                  pin_memory=False, drop_last=False, timeout=0,
                                  worker_init_fn=None, prefetch_factor=2,
                                  persistent_workers=False)
# del trainset

valid_labels = validset[1]
validset = DeepFakesDataset(validset[0], validset[1], config['model']['image-size'], mode='validation')
validloader = torch.utils.data.DataLoader(validset, batch_size=8, shuffle=True, #config['training']['bs']
                                      sampler=None,
                                      batch_sampler=None, num_workers=2, collate_fn=None,
                                      pin_memory=False, drop_last=False, timeout=0,
                                      worker_init_fn=None, prefetch_factor=2,
                                      persistent_workers=False)
# del validset

In [ ]:
# train
model = model.cuda()
counter = 0
not_improved_loss = 0
previous_loss = math.inf
print(starting_epoch)

23


# EffNet-B0

In [ ]:
import time
tr_loss = []
vl_loss = []
for t in range(starting_epoch, 30+1):
    if not_improved_loss == 5:
        break
    counter = 0

    total_loss = 0
    total_val_loss = 0

    bar = ChargingBar('EPOCH #' + str(t), max=(len(trainloader) * config['training']['bs']) + len(validloader))
    train_correct = 0
    positive = 0
    negative = 0
    gtp, gfp, gtn, gfn = 0, 0, 0, 0

    cur_step = 0
    begin = time.time()
    for index, (images, labels) in enumerate(trainloader):
        images = np.transpose(images, (0, 3, 1, 2))
        labels = labels.unsqueeze(1)
        images = images.cuda()

        y_pred = model(images)
        y_pred = y_pred.cpu()
        # print(y_pred, labels)
        loss = loss_fn(y_pred, labels.float())

        corrects, positive_class, negative_class, tpfptnfn = check_correct(y_pred, labels)
        train_correct += corrects
        positive += positive_class
        negative += negative_class
        gtp += tpfptnfn[0]
        gfp += tpfptnfn[1]
        gtn += tpfptnfn[2]
        gfn += tpfptnfn[3]
        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
        counter += 1
        total_loss += round(loss.item(), 2)

        if index % 25 == 0:  # Intermediate metrics print
            # interm = time.time()
            print(f"[{cur_step+1}/{25}]", "Loss: ", total_loss / counter, "Accuracy: ",
                  train_correct / (counter * config['training']['bs']), "Train 0s: ", negative, "Train 1s:",
                  positive)
            cur_step += 1
        for i in range(config['training']['bs']):
            bar.next()
    later = time.time()
    difference = int(later - begin)
    print(f"#{t+1}/{30} training time: {difference}s")
    cur_step = 0

    val_correct = 0
    val_positive = 0
    val_negative = 0
    val_counter = 0
    vgtp, vgfp, vgtn, vgfn = 0, 0, 0, 0
    train_correct /= train_samples
    total_loss /= counter
    for index, (val_images, val_labels) in enumerate(validloader):
        val_images = np.transpose(val_images, (0, 3, 1, 2))

        val_images = val_images.cuda()
        val_labels = val_labels.unsqueeze(1)
        val_pred = model(val_images)
        val_pred = val_pred.cpu()
        val_loss = loss_fn(val_pred, val_labels.float())
        total_val_loss += round(val_loss.item(), 2)
        corrects, positive_class, negative_class, tpfptnfn = check_correct(val_pred, val_labels)
        val_correct += corrects
        val_positive += positive_class
        val_counter += 1
        val_negative += negative_class
        vgtp += tpfptnfn[0]
        vgfp += tpfptnfn[1]
        vgtn += tpfptnfn[2]
        vgfn += tpfptnfn[3]
        bar.next()

    scheduler.step()
    bar.finish()

    total_val_loss /= val_counter
    val_correct /= validation_samples
    val_precision = vgtp / (vgtp + vgfp)
    val_recall = vgtp / (vgtp + vgfn)
    val_f1 = 2 * val_precision * val_recall / (val_precision + val_recall)
    if previous_loss <= total_val_loss:
        print("Validation loss did not improved")
        not_improved_loss += 1
    else:
        not_improved_loss = 0

    previous_loss = total_val_loss
    print("#" + str(t+1) + "/" + str(30) + " loss:" +
              str(total_loss) + " accuracy:" + str(train_correct) + " val_loss:" + str(
            total_val_loss) + " val_accuracy:" + str(val_correct) + " val_precision:" + str(val_precision) +
            " val_recall:" + str(val_recall) + " val_f1:" + str(val_f1) + " val_0s:" + str(val_negative) + "/" + str(
            np.count_nonzero(valid_labels == 0)) + " val_1s:" + str(val_positive) + "/" + str(
            np.count_nonzero(valid_labels == 1)) + "\n")

    tr_loss.append(total_loss)
    vl_loss.append(total_val_loss)

    if not os.path.exists('./checkpoint'):
        os.makedirs('./checkpoint')
    torch.save(model.state_dict(), os.path.join('./checkpoint',
                                                "efficientnetB" + str(0) + "_checkpoint" + str(
                                                    t) + "_" + "test"))
print(tr_loss)
print(vl_loss)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[1/25] Loss:  0.7 Accuracy:  0.53125 Train 0s:  19 Train 1s: 13
[2/25] Loss:  0.6957692307692308 Accuracy:  0.5024038461538461 Train 0s:  448 Train 1s: 384
[3/25] Loss:  0.6929411764705883 Accuracy:  0.5257352941176471 Train 0s:  897 Train 1s: 735
[4/25] Loss:  0.6922368421052629 Accuracy:  0.5263157894736842 Train 0s:  1305 Train 1s: 1127
[5/25] Loss:  0.6913861386138613 Accuracy:  0.5278465346534653 Train 0s:  1712 Train 1s: 1520
[6/25] Loss:  0.6922222222222221 Accuracy:  0.5245535714285714 Train 0s:  2103 Train 1s: 1929
[7/25] Loss:  0.6926490066225165 Accuracy:  0.5196605960264901 Train 0s:  2481 Train 1s: 2351
[8/25] Loss:  0.6928977272727271 Accuracy:  0.5202414772727273 Train 0s:  2838 Train 1s: 2794
[9/25] Loss:  0.6926368159203978 Accuracy:  0.5239427860696517 Train 0s:  3207 Train 1s: 3225
[10/25] Loss:  0.6928761061946903 Accuracy:  0.5232300884955752 Train 0s:  3564 Train 1s: 3668
[11/25] Loss:  0.6927091633466133 Accuracy:  0.5222858565737052 Train 0s:  3958 Train 1s: 407

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


#1/30 training time: 263s
#1/30 loss:0.6892800000000001 accuracy:0.5366768338416921 val_loss:0.6798412698412698 val_accuracy:0.5742871435717859 val_precision:0.5885167464114832 val_recall:0.4924924924924925 val_f1:0.5362397820163488 val_0s:1163/1000 val_1s:836/999

[1/25] Loss:  0.68 Accuracy:  0.59375 Train 0s:  20 Train 1s: 12
[2/25] Loss:  0.6811538461538462 Accuracy:  0.5625 Train 0s:  475 Train 1s: 357
[3/25] Loss:  0.6811764705882355 Accuracy:  0.5741421568627451 Train 0s:  924 Train 1s: 708
[4/25] Loss:  0.6836842105263159 Accuracy:  0.5682565789473685 Train 0s:  1362 Train 1s: 1070
[5/25] Loss:  0.6841584158415842 Accuracy:  0.5637376237623762 Train 0s:  1796 Train 1s: 1436
[6/25] Loss:  0.6845238095238099 Accuracy:  0.5595238095238095 Train 0s:  2207 Train 1s: 1825
[7/25] Loss:  0.6838410596026494 Accuracy:  0.5622930463576159 Train 0s:  2629 Train 1s: 2203
[8/25] Loss:  0.6833522727272731 Accuracy:  0.5662286931818182 Train 0s:  3069 Train 1s: 2563
[9/25] Loss:  0.68328358208

# EffNet-B0 + ViT

In [ ]:
import time
for t in range(starting_epoch, 15+1):
    if not_improved_loss == 5:
        break
    counter = 0

    total_loss = 0
    total_val_loss = 0

    bar = ChargingBar('EPOCH #' + str(t), max=(len(trainloader) * config['training']['bs']) + len(validloader))
    train_correct = 0
    positive = 0
    negative = 0
    gtp, gfp, gtn, gfn = 0, 0, 0, 0

    cur_step = 0
    begin = time.time()
    for index, (images, labels) in enumerate(trainloader):
        images = np.transpose(images, (0, 3, 1, 2))
        labels = labels.unsqueeze(1)
        images = images.cuda()

        y_pred = model(images)
        y_pred = y_pred.cpu()
        # print(y_pred, labels)
        loss = loss_fn(y_pred, labels.float())

        corrects, positive_class, negative_class, tpfptnfn = check_correct(y_pred, labels)
        train_correct += corrects
        positive += positive_class
        negative += negative_class
        gtp += tpfptnfn[0]
        gfp += tpfptnfn[1]
        gtn += tpfptnfn[2]
        gfn += tpfptnfn[3]
        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
        counter += 1
        total_loss += round(loss.item(), 2)

        if index % 25 == 0:  # Intermediate metrics print
            # interm = time.time()
            print(f"[{cur_step+1}/{25}]", "Loss: ", total_loss / counter, "Accuracy: ",
                  train_correct / (counter * config['training']['bs']), "Train 0s: ", negative, "Train 1s:",
                  positive)
            cur_step += 1
        for i in range(config['training']['bs']):
            bar.next()
    later = time.time()
    difference = int(later - begin)
    print(f"#{t+1}/{10} training time: {difference}s")
    cur_step = 0

    val_correct = 0
    val_positive = 0
    val_negative = 0
    val_counter = 0
    vgtp, vgfp, vgtn, vgfn = 0, 0, 0, 0
    train_correct /= train_samples
    total_loss /= counter
    for index, (val_images, val_labels) in enumerate(validloader):
        val_images = np.transpose(val_images, (0, 3, 1, 2))

        val_images = val_images.cuda()
        val_labels = val_labels.unsqueeze(1)
        val_pred = model(val_images)
        val_pred = val_pred.cpu()
        val_loss = loss_fn(val_pred, val_labels.float())
        total_val_loss += round(val_loss.item(), 2)
        corrects, positive_class, negative_class, tpfptnfn = check_correct(val_pred, val_labels)
        val_correct += corrects
        val_positive += positive_class
        val_counter += 1
        val_negative += negative_class
        vgtp += tpfptnfn[0]
        vgfp += tpfptnfn[1]
        vgtn += tpfptnfn[2]
        vgfn += tpfptnfn[3]
        bar.next()

    scheduler.step()
    bar.finish()

    total_val_loss /= val_counter
    val_correct /= validation_samples
    val_precision = vgtp / (vgtp + vgfp)
    val_recall = vgtp / (vgtp + vgfn)
    val_f1 = 2 * val_precision * val_recall / (val_precision + val_recall)
    if previous_loss <= total_val_loss:
        print("Validation loss did not improved")
        not_improved_loss += 1
    else:
        not_improved_loss = 0

    previous_loss = total_val_loss
    print("#" + str(t+1) + "/" + str(10) + " loss:" +
              str(total_loss) + " accuracy:" + str(train_correct) + " val_loss:" + str(
            total_val_loss) + " val_accuracy:" + str(val_correct) + " val_precision:" + str(val_precision) +
            " val_recall:" + str(val_recall) + " val_f1:" + str(val_f1) + " val_0s:" + str(val_negative) + "/" + str(
            np.count_nonzero(valid_labels == 0)) + " val_1s:" + str(val_positive) + "/" + str(
            np.count_nonzero(valid_labels == 1)) + "\n")

    if not os.path.exists('./checkpoint'):
        os.makedirs('./checkpoint')
    torch.save(model.state_dict(), os.path.join('./checkpoint',
                                                "efficientnetB" + str(0) + "_checkpoint" + str(
                                                    t) + "_" + "Test"))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[1/25] Loss:  0.72 Accuracy:  0.4375 Train 0s:  7 Train 1s: 25
[2/25] Loss:  0.7084615384615384 Accuracy:  0.47596153846153844 Train 0s:  352 Train 1s: 480
[3/25] Loss:  0.7037254901960788 Accuracy:  0.48284313725490197 Train 0s:  685 Train 1s: 947
[4/25] Loss:  0.7018421052631584 Accuracy:  0.4921875 Train 0s:  1148 Train 1s: 1284
[5/25] Loss:  0.7025742574257431 Accuracy:  0.48978960396039606 Train 0s:  1639 Train 1s: 1593
[6/25] Loss:  0.7004761904761907 Accuracy:  0.49776785714285715 Train 0s:  2063 Train 1s: 1969
[7/25] Loss:  0.7004635761589405 Accuracy:  0.49503311258278143 Train 0s:  2491 Train 1s: 2341
[8/25] Loss:  0.7003977272727273 Accuracy:  0.49573863636363635 Train 0s:  2927 Train 1s: 2705
[9/25] Loss:  0.6998507462686566 Accuracy:  0.49891169154228854 Train 0s:  3201 Train 1s: 3231
[10/25] Loss:  0.7001327433628315 Accuracy:  0.49668141592920356 Train 0s:  3687 Train 1s: 3545
[11/25] Loss:  0.6993227091633459 Accuracy:  0.49962649402390436 Train 0s:  4171 Train 1s: 3861

In [ ]:
import time
tr_loss = []
vl_loss = []
for t in range(starting_epoch, 30+1):
    if not_improved_loss == 5:
        break
    counter = 0

    total_loss = 0
    total_val_loss = 0

    bar = ChargingBar('EPOCH #' + str(t), max=(len(trainloader) * config['training']['bs']) + len(validloader))
    train_correct = 0
    positive = 0
    negative = 0
    gtp, gfp, gtn, gfn = 0, 0, 0, 0

    cur_step = 0
    begin = time.time()
    for index, (images, labels) in enumerate(trainloader):
        images = np.transpose(images, (0, 3, 1, 2))
        labels = labels.unsqueeze(1)
        images = images.cuda()

        y_pred = model(images)
        y_pred = y_pred.cpu()
        # print(y_pred, labels)
        loss = loss_fn(y_pred, labels.float())

        corrects, positive_class, negative_class, tpfptnfn = check_correct(y_pred, labels)
        train_correct += corrects
        positive += positive_class
        negative += negative_class
        gtp += tpfptnfn[0]
        gfp += tpfptnfn[1]
        gtn += tpfptnfn[2]
        gfn += tpfptnfn[3]
        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
        counter += 1
        total_loss += round(loss.item(), 2)

        if index % 25 == 0:  # Intermediate metrics print
            # interm = time.time()
            print(f"[{cur_step+1}/{25}]", "Loss: ", total_loss / counter, "Accuracy: ",
                  train_correct / (counter * config['training']['bs']), "Train 0s: ", negative, "Train 1s:",
                  positive)
            cur_step += 1
        for i in range(config['training']['bs']):
            bar.next()
    later = time.time()
    difference = int(later - begin)
    print(f"#{t+1}/{10} training time: {difference}s")
    cur_step = 0

    val_correct = 0
    val_positive = 0
    val_negative = 0
    val_counter = 0
    vgtp, vgfp, vgtn, vgfn = 0, 0, 0, 0
    train_correct /= train_samples
    total_loss /= counter
    for index, (val_images, val_labels) in enumerate(validloader):
        val_images = np.transpose(val_images, (0, 3, 1, 2))

        val_images = val_images.cuda()
        val_labels = val_labels.unsqueeze(1)
        val_pred = model(val_images)
        val_pred = val_pred.cpu()
        val_loss = loss_fn(val_pred, val_labels.float())
        total_val_loss += round(val_loss.item(), 2)
        corrects, positive_class, negative_class, tpfptnfn = check_correct(val_pred, val_labels)
        val_correct += corrects
        val_positive += positive_class
        val_counter += 1
        val_negative += negative_class
        vgtp += tpfptnfn[0]
        vgfp += tpfptnfn[1]
        vgtn += tpfptnfn[2]
        vgfn += tpfptnfn[3]
        bar.next()

    scheduler.step()
    bar.finish()

    total_val_loss /= val_counter
    val_correct /= validation_samples
    val_precision = vgtp / (vgtp + vgfp)
    val_recall = vgtp / (vgtp + vgfn)
    val_f1 = 2 * val_precision * val_recall / (val_precision + val_recall)
    if previous_loss <= total_val_loss:
        print("Validation loss did not improved")
        not_improved_loss += 1
    else:
        not_improved_loss = 0

    previous_loss = total_val_loss
    print("#" + str(t) + "/" + str(30) + " loss:" +
              str(total_loss) + " accuracy:" + str(train_correct) + " val_loss:" + str(
            total_val_loss) + " val_accuracy:" + str(val_correct) + " val_precision:" + str(val_precision) +
            " val_recall:" + str(val_recall) + " val_f1:" + str(val_f1) + " val_0s:" + str(val_negative) + "/" + str(
            np.count_nonzero(valid_labels == 0)) + " val_1s:" + str(val_positive) + "/" + str(
            np.count_nonzero(valid_labels == 1)) + "\n")

    tr_loss.append(total_loss)
    vl_loss.append(total_val_loss)

    if not os.path.exists('./checkpoint'):
        os.makedirs('./checkpoint')
    torch.save(model.state_dict(), os.path.join('./checkpoint',
                                                "efficientnetB" + str(0) + "_checkpoint" + str(
                                                    t) + "_" + "test"))
print(tr_loss)
print(vl_loss)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[1/25] Loss:  0.24 Accuracy:  0.90625 Train 0s:  16 Train 1s: 16
[2/25] Loss:  0.3688461538461537 Accuracy:  0.8509615384615384 Train 0s:  419 Train 1s: 413
[3/25] Loss:  0.3735294117647059 Accuracy:  0.8388480392156863 Train 0s:  822 Train 1s: 810
[4/25] Loss:  0.38513157894736844 Accuracy:  0.8273026315789473 Train 0s:  1212 Train 1s: 1220
[5/25] Loss:  0.3814851485148514 Accuracy:  0.8285891089108911 Train 0s:  1632 Train 1s: 1600
[6/25] Loss:  0.38777777777777767 Accuracy:  0.8249007936507936 Train 0s:  1999 Train 1s: 2033
[7/25] Loss:  0.3850993377483442 Accuracy:  0.8271937086092715 Train 0s:  2400 Train 1s: 2432
[8/25] Loss:  0.38982954545454535 Accuracy:  0.8235085227272727 Train 0s:  2825 Train 1s: 2807
[9/25] Loss:  0.3942288557213928 Accuracy:  0.8212064676616916 Train 0s:  3258 Train 1s: 3174
[10/25] Loss:  0.3927876106194689 Accuracy:  0.8210730088495575 Train 0s:  3679 Train 1s: 3553
[11/25] Loss:  0.39521912350597604 Accuracy:  0.8199701195219123 Train 0s:  4101 Train 1s

# EffNet-B1 + ViT

In [ ]:
# 1 ~ 12 epoch
import time
tr_loss = []
vl_loss = []
tr_acc = []
vl_acc = []
vl_precision = []
vl_recall = []
vl_f1 = []

for t in range(starting_epoch, 30):
    if not_improved_loss == 5:
        break
    counter = 0

    total_loss = 0
    total_val_loss = 0

    bar = ChargingBar('EPOCH #' + str(t), max=(len(trainloader) * 4) + len(validloader))
    train_correct = 0
    positive = 0
    negative = 0
    gtp, gfp, gtn, gfn = 0, 0, 0, 0

    cur_step = 0
    begin = time.time()
    for index, (images, labels) in enumerate(trainloader):
        images = np.transpose(images, (0, 3, 1, 2))
        labels = labels.unsqueeze(1)
        images = images.cuda()

        y_pred = model(images)
        y_pred = y_pred.cpu()
        # print(y_pred, labels)
        loss = loss_fn(y_pred, labels.float())

        corrects, positive_class, negative_class, tpfptnfn = check_correct(y_pred, labels)
        train_correct += corrects
        positive += positive_class
        negative += negative_class
        gtp += tpfptnfn[0]
        gfp += tpfptnfn[1]
        gtn += tpfptnfn[2]
        gfn += tpfptnfn[3]
        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
        counter += 1
        total_loss += round(loss.item(), 2)

        if index % 50 == 0:  # Intermediate metrics print
            # interm = time.time()
            print(f"[{cur_step+1}/{50}]", "Loss: ", total_loss / counter, "Accuracy: ",
                  train_correct / (counter * 8), "Train 0s: ", negative, "Train 1s:",
                  positive)
            cur_step += 1
        for i in range(8):
            bar.next()
    later = time.time()
    difference = int(later - begin)
    print(f"#{t+1}/{30} training time: {difference}s")
    cur_step = 0

    val_correct = 0
    val_positive = 0
    val_negative = 0
    val_counter = 0
    vgtp, vgfp, vgtn, vgfn = 0, 0, 0, 0
    train_correct /= train_samples
    total_loss /= counter

    for index, (val_images, val_labels) in enumerate(validloader):
        val_images = np.transpose(val_images, (0, 3, 1, 2))

        val_images = val_images.cuda()
        val_labels = val_labels.unsqueeze(1)
        val_pred = model(val_images)
        val_pred = val_pred.cpu()
        val_loss = loss_fn(val_pred, val_labels.float())
        total_val_loss += round(val_loss.item(), 2)
        corrects, positive_class, negative_class, tpfptnfn = check_correct(val_pred, val_labels)
        val_correct += corrects
        val_positive += positive_class
        val_counter += 1
        val_negative += negative_class
        vgtp += tpfptnfn[0]
        vgfp += tpfptnfn[1]
        vgtn += tpfptnfn[2]
        vgfn += tpfptnfn[3]
        bar.next()

    scheduler.step()
    bar.finish()

    total_val_loss /= val_counter
    val_correct /= validation_samples
    val_precision = vgtp / (vgtp + vgfp)
    val_recall = vgtp / (vgtp + vgfn)
    val_f1 = 2 * val_precision * val_recall / (val_precision + val_recall)
    if previous_loss <= total_val_loss:
        print("Validation loss did not improved")
        not_improved_loss += 1
    else:
        not_improved_loss = 0

    previous_loss = total_val_loss
    print("#" + str(t+1) + "/" + str(30) + " loss:" +
              str(total_loss) + " accuracy:" + str(train_correct) + " val_loss:" + str(
            total_val_loss) + " val_accuracy:" + str(val_correct) + " val_precision:" + str(val_precision) +
            " val_recall:" + str(val_recall) + " val_f1:" + str(val_f1) + " val_0s:" + str(val_negative) + "/" + str(
            np.count_nonzero(valid_labels == 0)) + " val_1s:" + str(val_positive) + "/" + str(
            np.count_nonzero(valid_labels == 1)) + "\n")

    tr_loss.append(total_loss)
    vl_loss.append(total_val_loss)
    tr_acc.append(train_correct)
    vl_acc.append(val_correct)
    vl_precision.append(val_precision)
    vl_recall.append(val_recall)
    vl_f1.append(val_f1)

    if not os.path.exists('./checkpoint'):
        os.makedirs('./checkpoint')
    torch.save(model.state_dict(), os.path.join('./checkpoint',
                                                "efficientnetB" + str(7) + "_checkpoint" + str(
                                                    t) + "_" + "test"))
print(tr_loss)
print(vl_loss)
print(tr_acc)
print(vl_acc)
print(vl_precision)
print(vl_recall)
print(vl_f1)

/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


[1/50] Loss:  0.71 Accuracy:  0.625 Train 0s:  0 Train 1s: 8
[2/50] Loss:  0.7033333333333333 Accuracy:  0.5220588235294118 Train 0s:  183 Train 1s: 225
[3/50] Loss:  0.7117821782178217 Accuracy:  0.5037128712871287 Train 0s:  401 Train 1s: 407
[4/50] Loss:  0.7102649006622518 Accuracy:  0.4917218543046358 Train 0s:  538 Train 1s: 670
[5/50] Loss:  0.7102985074626862 Accuracy:  0.48880597014925375 Train 0s:  716 Train 1s: 892
[6/50] Loss:  0.7089243027888446 Accuracy:  0.499003984063745 Train 0s:  901 Train 1s: 1107
[7/50] Loss:  0.7090697674418603 Accuracy:  0.5004152823920266 Train 0s:  1033 Train 1s: 1375
[8/50] Loss:  0.709202279202279 Accuracy:  0.5035612535612536 Train 0s:  1205 Train 1s: 1603
[9/50] Loss:  0.7091022443890275 Accuracy:  0.5 Train 0s:  1455 Train 1s: 1753
[10/50] Loss:  0.7092682926829271 Accuracy:  0.5002771618625277 Train 0s:  1623 Train 1s: 1985
[11/50] Loss:  0.708642714570859 Accuracy:  0.5012475049900199 Train 0s:  1877 Train 1s: 2131
[12/50] Loss:  0.708874

In [ ]:
# 13 ~ 23 epoch
import time
tr_loss = []
vl_loss = []
tr_acc = []
vl_acc = []
vl_precision = []
vl_recall = []
vl_f1 = []

for t in range(starting_epoch, 30):
    if not_improved_loss == 5:
        break
    counter = 0

    total_loss = 0
    total_val_loss = 0

    bar = ChargingBar('EPOCH #' + str(t), max=(len(trainloader) * 4) + len(validloader))
    train_correct = 0
    positive = 0
    negative = 0
    gtp, gfp, gtn, gfn = 0, 0, 0, 0

    cur_step = 0
    begin = time.time()
    for index, (images, labels) in enumerate(trainloader):
        images = np.transpose(images, (0, 3, 1, 2))
        labels = labels.unsqueeze(1)
        images = images.cuda()

        y_pred = model(images)
        y_pred = y_pred.cpu()
        # print(y_pred, labels)
        loss = loss_fn(y_pred, labels.float())

        corrects, positive_class, negative_class, tpfptnfn = check_correct(y_pred, labels)
        train_correct += corrects
        positive += positive_class
        negative += negative_class
        gtp += tpfptnfn[0]
        gfp += tpfptnfn[1]
        gtn += tpfptnfn[2]
        gfn += tpfptnfn[3]
        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
        counter += 1
        total_loss += round(loss.item(), 2)

        if index % 50 == 0:  # Intermediate metrics print
            # interm = time.time()
            print(f"[{cur_step+1}/{50}]", "Loss: ", total_loss / counter, "Accuracy: ",
                  train_correct / (counter * 8), "Train 0s: ", negative, "Train 1s:",
                  positive)
            cur_step += 1
        for i in range(8):
            bar.next()
    later = time.time()
    difference = int(later - begin)
    print(f"#{t+1}/{30} training time: {difference}s")
    cur_step = 0

    val_correct = 0
    val_positive = 0
    val_negative = 0
    val_counter = 0
    vgtp, vgfp, vgtn, vgfn = 0, 0, 0, 0
    train_correct /= train_samples
    total_loss /= counter

    for index, (val_images, val_labels) in enumerate(validloader):
        val_images = np.transpose(val_images, (0, 3, 1, 2))

        val_images = val_images.cuda()
        val_labels = val_labels.unsqueeze(1)
        val_pred = model(val_images)
        val_pred = val_pred.cpu()
        val_loss = loss_fn(val_pred, val_labels.float())
        total_val_loss += round(val_loss.item(), 2)
        corrects, positive_class, negative_class, tpfptnfn = check_correct(val_pred, val_labels)
        val_correct += corrects
        val_positive += positive_class
        val_counter += 1
        val_negative += negative_class
        vgtp += tpfptnfn[0]
        vgfp += tpfptnfn[1]
        vgtn += tpfptnfn[2]
        vgfn += tpfptnfn[3]
        bar.next()

    scheduler.step()
    bar.finish()

    total_val_loss /= val_counter
    val_correct /= validation_samples
    val_precision = vgtp / (vgtp + vgfp)
    val_recall = vgtp / (vgtp + vgfn)
    val_f1 = 2 * val_precision * val_recall / (val_precision + val_recall)
    if previous_loss <= total_val_loss:
        print("Validation loss did not improved")
        not_improved_loss += 1
    else:
        not_improved_loss = 0

    previous_loss = total_val_loss
    print("#" + str(t+1) + "/" + str(30) + " loss:" +
              str(total_loss) + " accuracy:" + str(train_correct) + " val_loss:" + str(
            total_val_loss) + " val_accuracy:" + str(val_correct) + " val_precision:" + str(val_precision) +
            " val_recall:" + str(val_recall) + " val_f1:" + str(val_f1) + " val_0s:" + str(val_negative) + "/" + str(
            np.count_nonzero(valid_labels == 0)) + " val_1s:" + str(val_positive) + "/" + str(
            np.count_nonzero(valid_labels == 1)) + "\n")

    tr_loss.append(total_loss)
    vl_loss.append(total_val_loss)
    tr_acc.append(train_correct)
    vl_acc.append(val_correct)
    vl_precision.append(val_precision)
    vl_recall.append(val_recall)
    vl_f1.append(val_f1)

    if not os.path.exists('./checkpoint'):
        os.makedirs('./checkpoint')
    torch.save(model.state_dict(), os.path.join('./checkpoint',
                                                "efficientnetB" + str(7) + "_checkpoint" + str(
                                                    t) + "_" + "test"))
print(tr_loss)
print(vl_loss)
print(tr_acc)
print(vl_acc)
print(vl_precision)
print(vl_recall)
print(vl_f1)

/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


[1/50] Loss:  0.96 Accuracy:  0.625 Train 0s:  4 Train 1s: 4
[2/50] Loss:  0.39607843137254906 Accuracy:  0.8186274509803921 Train 0s:  207 Train 1s: 201
[3/50] Loss:  0.41207920792079217 Accuracy:  0.806930693069307 Train 0s:  426 Train 1s: 382
[4/50] Loss:  0.39794701986754977 Accuracy:  0.8211920529801324 Train 0s:  636 Train 1s: 572
[5/50] Loss:  0.40487562189054765 Accuracy:  0.8208955223880597 Train 0s:  826 Train 1s: 782
[6/50] Loss:  0.3995219123505978 Accuracy:  0.8237051792828686 Train 0s:  1027 Train 1s: 981
[7/50] Loss:  0.39478405315614645 Accuracy:  0.8255813953488372 Train 0s:  1235 Train 1s: 1173
[8/50] Loss:  0.3918518518518519 Accuracy:  0.8254985754985755 Train 0s:  1452 Train 1s: 1356
[9/50] Loss:  0.3838653366583543 Accuracy:  0.8288653366583542 Train 0s:  1646 Train 1s: 1562
[10/50] Loss:  0.3799778270509979 Accuracy:  0.8317627494456763 Train 0s:  1832 Train 1s: 1776
[11/50] Loss:  0.3792015968063872 Accuracy:  0.8323353293413174 Train 0s:  2038 Train 1s: 1970
[1

KeyboardInterrupt: 

In [ ]:
# 13 ~ 23 epoch
print(tr_loss)
print(vl_loss)
print(tr_acc)
print(vl_acc)
print(vl_precision)
print(vl_recall)
print(vl_f1)

[0.3055039999999997, 0.29692000000000046, 0.298908, 0.29153999999999985, 0.2860959999999995, 0.2753599999999994, 0.2781200000000001]
[0.27072000000000007, 0.26872, 0.2589599999999998, 0.2653599999999999, 0.2564800000000003, 0.2686000000000002, 0.25384]
[0.8655432771638581, 0.8693934696734836, 0.8700435021751087, 0.8691934596729837, 0.8772438621931097, 0.8780439021951097, 0.88064403220161]
[0.8859429714857429, 0.8884442221110556, 0.8919459729864933, 0.886943471735868, 0.8989494747373686, 0.8849424712356178, 0.903951975987994]
[0.8592730661696178, 0.885685884691849, 0.9142857142857143, 0.8861138861138861, 0.9078812691914022, 0.8736637512147716, 0.9113149847094801]
[0.9229229229229229, 0.8918918918918919, 0.8648648648648649, 0.8878878878878879, 0.8878878878878879, 0.8998998998998999, 0.8948948948948949]
[0.88996138996139, 0.8887780548628429, 0.888888888888889, 0.887, 0.8977732793522268, 0.8865877712031558, 0.9030303030303031]


In [ ]:
# 24 ~  epoch
import time
tr_loss = []
vl_loss = []
tr_acc = []
vl_acc = []
vl_precision = []
vl_recall = []
vl_f1 = []

for t in range(starting_epoch, 30):
    if not_improved_loss == 5:
        break
    counter = 0

    total_loss = 0
    total_val_loss = 0

    bar = ChargingBar('EPOCH #' + str(t), max=(len(trainloader) * 4) + len(validloader))
    train_correct = 0
    positive = 0
    negative = 0
    gtp, gfp, gtn, gfn = 0, 0, 0, 0

    cur_step = 0
    begin = time.time()
    for index, (images, labels) in enumerate(trainloader):
        images = np.transpose(images, (0, 3, 1, 2))
        labels = labels.unsqueeze(1)
        images = images.cuda()

        y_pred = model(images)
        y_pred = y_pred.cpu()
        # print(y_pred, labels)
        loss = loss_fn(y_pred, labels.float())

        corrects, positive_class, negative_class, tpfptnfn = check_correct(y_pred, labels)
        train_correct += corrects
        positive += positive_class
        negative += negative_class
        gtp += tpfptnfn[0]
        gfp += tpfptnfn[1]
        gtn += tpfptnfn[2]
        gfn += tpfptnfn[3]
        optimizer.zero_grad()

        loss.backward()

        optimizer.step()
        counter += 1
        total_loss += round(loss.item(), 2)

        if index % 50 == 0:  # Intermediate metrics print
            # interm = time.time()
            print(f"[{cur_step+1}/{50}]", "Loss: ", total_loss / counter, "Accuracy: ",
                  train_correct / (counter * 8), "Train 0s: ", negative, "Train 1s:",
                  positive)
            cur_step += 1
        for i in range(8):
            bar.next()
    later = time.time()
    difference = int(later - begin)
    print(f"#{t+1}/{30} training time: {difference}s")
    cur_step = 0

    val_correct = 0
    val_positive = 0
    val_negative = 0
    val_counter = 0
    vgtp, vgfp, vgtn, vgfn = 0, 0, 0, 0
    train_correct /= train_samples
    total_loss /= counter

    for index, (val_images, val_labels) in enumerate(validloader):
        val_images = np.transpose(val_images, (0, 3, 1, 2))

        val_images = val_images.cuda()
        val_labels = val_labels.unsqueeze(1)
        val_pred = model(val_images)
        val_pred = val_pred.cpu()
        val_loss = loss_fn(val_pred, val_labels.float())
        total_val_loss += round(val_loss.item(), 2)
        corrects, positive_class, negative_class, tpfptnfn = check_correct(val_pred, val_labels)
        val_correct += corrects
        val_positive += positive_class
        val_counter += 1
        val_negative += negative_class
        vgtp += tpfptnfn[0]
        vgfp += tpfptnfn[1]
        vgtn += tpfptnfn[2]
        vgfn += tpfptnfn[3]
        bar.next()

    scheduler.step()
    bar.finish()

    total_val_loss /= val_counter
    val_correct /= validation_samples
    val_precision = vgtp / (vgtp + vgfp)
    val_recall = vgtp / (vgtp + vgfn)
    val_f1 = 2 * val_precision * val_recall / (val_precision + val_recall)
    if previous_loss <= total_val_loss:
        print("Validation loss did not improved")
        not_improved_loss += 1
    else:
        not_improved_loss = 0

    previous_loss = total_val_loss
    print("#" + str(t+1) + "/" + str(30) + " loss:" +
              str(total_loss) + " accuracy:" + str(train_correct) + " val_loss:" + str(
            total_val_loss) + " val_accuracy:" + str(val_correct) + " val_precision:" + str(val_precision) +
            " val_recall:" + str(val_recall) + " val_f1:" + str(val_f1) + " val_0s:" + str(val_negative) + "/" + str(
            np.count_nonzero(valid_labels == 0)) + " val_1s:" + str(val_positive) + "/" + str(
            np.count_nonzero(valid_labels == 1)) + "\n")

    tr_loss.append(total_loss)
    vl_loss.append(total_val_loss)
    tr_acc.append(train_correct)
    vl_acc.append(val_correct)
    vl_precision.append(val_precision)
    vl_recall.append(val_recall)
    vl_f1.append(val_f1)

    if not os.path.exists('./checkpoint'):
        os.makedirs('./checkpoint')
    torch.save(model.state_dict(), os.path.join('./checkpoint',
                                                "efficientnetB" + str(7) + "_checkpoint" + str(
                                                    t) + "_" + "test"))
print(tr_loss)
print(vl_loss)
print(tr_acc)
print(vl_acc)
print(vl_precision)
print(vl_recall)
print(vl_f1)

/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


[1/50] Loss:  0.56 Accuracy:  0.625 Train 0s:  6 Train 1s: 2
[2/50] Loss:  0.2968627450980393 Accuracy:  0.8627450980392157 Train 0s:  219 Train 1s: 189
[3/50] Loss:  0.3104950495049506 Accuracy:  0.8576732673267327 Train 0s:  418 Train 1s: 390
[4/50] Loss:  0.3072847682119207 Accuracy:  0.8625827814569537 Train 0s:  617 Train 1s: 591
[5/50] Loss:  0.30935323383084584 Accuracy:  0.8619402985074627 Train 0s:  834 Train 1s: 774
[6/50] Loss:  0.3070916334661355 Accuracy:  0.8630478087649402 Train 0s:  1034 Train 1s: 974
[7/50] Loss:  0.31322259136212627 Accuracy:  0.8625415282392026 Train 0s:  1251 Train 1s: 1157
[8/50] Loss:  0.31532763532763525 Accuracy:  0.8593304843304843 Train 0s:  1458 Train 1s: 1350
[9/50] Loss:  0.3078304239401495 Accuracy:  0.8619077306733167 Train 0s:  1650 Train 1s: 1558
[10/50] Loss:  0.31053215077605323 Accuracy:  0.8611419068736141 Train 0s:  1839 Train 1s: 1769
[11/50] Loss:  0.3090419161676647 Accuracy:  0.8617764471057884 Train 0s:  2022 Train 1s: 1986
[1

In [ ]:
# 24 ~  epoch
print(tr_loss)
print(vl_loss)
print(tr_acc)
print(vl_acc)
print(vl_precision)
print(vl_recall)
print(vl_f1)

[0.3055039999999997, 0.29692000000000046, 0.298908, 0.29153999999999985, 0.2860959999999995, 0.2753599999999994, 0.2781200000000001]
[0.27072000000000007, 0.26872, 0.2589599999999998, 0.2653599999999999, 0.2564800000000003, 0.2686000000000002, 0.25384]
[0.8655432771638581, 0.8693934696734836, 0.8700435021751087, 0.8691934596729837, 0.8772438621931097, 0.8780439021951097, 0.88064403220161]
[0.8859429714857429, 0.8884442221110556, 0.8919459729864933, 0.886943471735868, 0.8989494747373686, 0.8849424712356178, 0.903951975987994]
[0.8592730661696178, 0.885685884691849, 0.9142857142857143, 0.8861138861138861, 0.9078812691914022, 0.8736637512147716, 0.9113149847094801]
[0.9229229229229229, 0.8918918918918919, 0.8648648648648649, 0.8878878878878879, 0.8878878878878879, 0.8998998998998999, 0.8948948948948949]
[0.88996138996139, 0.8887780548628429, 0.888888888888889, 0.887, 0.8977732793522268, 0.8865877712031558, 0.9030303030303031]
